## Customer Segmentation using Unsupervised Machine Learning 

### 1. Project Background

### 2. Load Data & Necessary Libraries

### 3. Data Cleaning & Preparation

### 4. Exploratory Data Analysis

### 5. Data Preprocessing 

### 6. Clustering algorithms

##### 6.1 Customer Segmentation with Hierarchical clustering

##### 6.1 Customer Segmentation with K-means clustering

#### 6.3 Customer Segmentation with Gaussian Mixture Models

##### 6.4 Customer Segmentation with DBSCAN